In [ ]:
import WeatherRoutingTool

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv() 

In [ ]:
%%bash
python cli.py -f config.template.json

In [ ]:
import os
import json
from ipyleaflet import Map, GeoJSON

m = Map(center=[39.926688, 7.930542], zoom=5.5)
# Note the coordinate order: [latitude, longitude]

with open("min_time_route.json") as f:
    data = json.load(f)

# Add the GeoJSON layer
geo_json = GeoJSON(
    data=data
)

# Add the GeoJSON layer to the map
m.add_layer(geo_json)
m

In [ ]:
resize_Images_WRT

In [ ]:
import os
from PIL import Image
from IPython.display import display, clear_output
import ipywidgets as widgets

# Pfad zum Bilderordner
image_folder = "Images-WRT"
image_files = sorted([
    f for f in os.listdir(image_folder)
    if f.lower().endswith('.png')
])

# Output-Widget für das Bild (damit es im Layout steuerbar ist)
image_output = widgets.Output()

# Funktion zur Bildanzeige
def show_image(index):
    image_path = os.path.join(image_folder, image_files[index])
    with image_output:
        image_output.clear_output(wait=True)
        img = Image.open(image_path)
        display(img)

# Vertikaler Slider
slider = widgets.IntSlider(
    value=0,
    min=0,
    max=len(image_files) - 1,
    step=1,
    orientation='vertical',
    description='routing step:',
    continuous_update=True,
    layout=widgets.Layout(height='300px')
)

# Callback bei Änderung
slider.observe(lambda change: show_image(change['new']), names='value')

# Layout: Slider links, Bild rechts
ui = widgets.HBox([slider,image_output], layout=widgets.Layout(align_items='center'))

# Erstes Bild anzeigen
show_image(0)

# Anzeige
display(ui)

In [ ]:
import json
from ipyleaflet import Map, Marker, AwesomeIcon
import ipywidgets as widgets
from ipywidgets import Button

icon1 = AwesomeIcon(
    name='map-pin',       # Icon von FontAwesome
    marker_color='green',       # Markerfarbe
    icon_color='white',       # Farbe des Symbols
    spin=False
)

icon2 = AwesomeIcon(
    name='map-pin',       # Icon von FontAwesome
    marker_color='red',       # Markerfarbe
    icon_color='white',       # Farbe des Symbols
    spin=False
)

center = ([39.926688, 7.930542])

m = Map(center=center, zoom=6)

marker1 = Marker(location=center, draggable=True, icon=icon1)
marker2 = Marker(location=center, draggable=True, icon=icon2)

m.add(marker1);
m.add(marker2);

button= widgets.Button(description="start routing")

def on_button_clicked()
    with open('config.template.json', 'r+') as f:
    data = json.load(f)
    data['DEFAULT_ROUTE'] = [marker1.location, marker2.location]
    with open('config.template.json', 'w') as file: 
    json.dump(data, file, indent=4)
    %run cli.py config.template.json

display(m)

In [ ]:
import json

with open('config.template.json', 'r+') as f:
    data = json.load(f)
    data['DEFAULT_ROUTE'] = [43.304959, 5.334957,40.806764, 14.282018]
with open('config.template.json', 'w') as file: 
    json.dump(data, file, indent=4)